<h1>Exercise 23</h1>
<p>4 SQL problems with high complexity.</p>
<ul>
<li>Problem 1: Write a query to get the list of students who scored above the average marks in each subject.</li>
<li>Problem 2: Write a query to get the percentage of students who scored more than 90 in any subject amongst total students.</li>
<li>Problem 3: Write a query to get the second highest and second lowest marks for each subject</li>
<li>Problem 4: For each student and test, identify if their marks increased or decreased from the previous test.</li>

</ul>
<strong>Expected outcomes:</strong>
<br>
<p>Problem 1:</p>
<img src="Day-30-Expected_Output1.png" width="450">
<br>
<p>Problem 2:</p>
<img src="Day-30-Expected_Output2.png" width="200">
<br>
<p>Problem 3:</p>
<img src="Day-30-Expected_Output3.png" width="450">
<br>
<p>Problem 4:</p>
<img src="Day-30-Expected_Output4.png" width="450">

In [1]:
import os
import sqlite3
import pandas as pd

db_name = "exercise30.db"
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS students(
    studentid INT,
    studentname TEXT,
    subject TEXT,
    marks INT,
    testid INT,
    testdate DATE
)""")
conn.commit()

students_data = [
    (2, 'Max Ruin', 'Subject1', 63, 1, '2022-01-02'),
    (3, 'Arnold', 'Subject1', 95, 1, '2022-01-02'),
    (4, 'Krish Star', 'Subject1', 61, 1, '2022-01-02'),
    (5, 'John Mike', 'Subject1', 91, 1, '2022-01-02'),
    (4, 'Krish Star', 'Subject2', 71, 1, '2022-01-02'),
    (3, 'Arnold', 'Subject2', 32, 1, '2022-01-02'),
    (5, 'John Mike', 'Subject2', 61, 2, '2022-11-02'),
    (1, 'John Deo', 'Subject2', 60, 1, '2022-01-02'),
    (2, 'Max Ruin', 'Subject2', 84, 1, '2022-01-02'),
    (2, 'Max Ruin', 'Subject3', 29, 3, '2022-01-03'),
    (5, 'John Mike', 'Subject3', 98, 2, '2022-11-02')
]

cursor.executemany("INSERT INTO students VALUES(?,?,?,?,?,?)", students_data)
conn.commit()

In [2]:
firstq = "WITH avg_cte AS (SELECT subject, AVG(marks) AS avg_marks FROM students GROUP BY subject) SELECT s.*, ac.* FROM students s INNER JOIN avg_cte ac ON s.subject = ac.subject WHERE s.marks > ac.avg_marks;"
df = pd.read_sql(firstq, conn)
print(df)

   studentid studentname   subject  marks  testid    testdate   subject  \
0          3      Arnold  Subject1     95       1  2022-01-02  Subject1   
1          5   John Mike  Subject1     91       1  2022-01-02  Subject1   
2          4  Krish Star  Subject2     71       1  2022-01-02  Subject2   
3          2    Max Ruin  Subject2     84       1  2022-01-02  Subject2   
4          5   John Mike  Subject3     98       2  2022-11-02  Subject3   

   avg_marks  
0       77.5  
1       77.5  
2       61.6  
3       61.6  
4       63.5  


In [3]:
secondq = "SELECT COUNT(DISTINCT CASE WHEN marks > 90 THEN studentid ELSE NULL END) * 1.0 / COUNT(DISTINCT studentid) * 100 AS perc FROM students;"
df = pd.read_sql(secondq, conn)
print(df)

   perc
0  40.0


In [4]:
thirdq = "SELECT subject, SUM(CASE WHEN rnk_desc = 2 THEN marks ELSE NULL END) AS second_highest_marks, SUM(CASE WHEN rnk_asc = 2 THEN marks ELSE NULL END) AS second_lowest_marks FROM (SELECT subject, marks, RANK() OVER(PARTITION BY subject ORDER BY marks ASC) AS rnk_asc, RANK() OVER(PARTITION BY subject ORDER BY marks DESC) AS rnk_desc FROM students) A GROUP BY subject;"
df = pd.read_sql(thirdq, conn)
print(df)

    subject  second_highest_marks  second_lowest_marks
0  Subject1                    91                   63
1  Subject2                    71                   60
2  Subject3                    29                   98


In [5]:
fourthq = "SELECT *, CASE WHEN marks > prev_marks THEN 'inc' WHEN marks < prev_marks THEN 'dec' ELSE NULL END AS statys FROM (SELECT *, LAG(marks, 1) OVER(PARTITION BY studentid ORDER BY testdate, subject) AS prev_marks FROM students) A;"
df = pd.read_sql(fourthq, conn)
print(df)
conn.close()

    studentid studentname   subject  marks  testid    testdate  prev_marks  \
0           1    John Deo  Subject2     60       1  2022-01-02         NaN   
1           2    Max Ruin  Subject1     63       1  2022-01-02         NaN   
2           2    Max Ruin  Subject2     84       1  2022-01-02        63.0   
3           2    Max Ruin  Subject3     29       3  2022-01-03        84.0   
4           3      Arnold  Subject1     95       1  2022-01-02         NaN   
5           3      Arnold  Subject2     32       1  2022-01-02        95.0   
6           4  Krish Star  Subject1     61       1  2022-01-02         NaN   
7           4  Krish Star  Subject2     71       1  2022-01-02        61.0   
8           5   John Mike  Subject1     91       1  2022-01-02         NaN   
9           5   John Mike  Subject2     61       2  2022-11-02        91.0   
10          5   John Mike  Subject3     98       2  2022-11-02        61.0   

   statys  
0    None  
1    None  
2     inc  
3     dec  
4  